# Collecting tweets using tweeepy and aylienapiclient

Required Packages:
1. tweepy
2. csv
3. matplotlib
4. aylien-apiclient==0.1.0  #!pip install aylien-apiclient==0.1.0

### For collecting tweets with twitter API i.e. tweepy, you need to have api credentials and for getting credentials you need to sign in with twitter. Same for AylienApiClient


In [1]:
import sys
import csv
import tweepy
import matplotlib.pyplot as plt
import time

from collections import Counter
from aylienapiclient import textapi

if sys.version_info[0] < 3:
   input = raw_input

## Twitter credentials
consumer_key = "buD36prPWXKOHJhtsq1yhj4d5"
consumer_secret = "LsjuTUH1mchDCfxLHcZkUzPWFvikZLAAj5uaLuxZZ0lTtPMTrV"
access_token = "973597389226471424-4HI6QmITahXeyfg2U9cos1wJhtwUZdb"
access_token_secret = "YjtNFgmWcF5mUghtMCrwQADk6c2wToT84gSl1MGqh3Pvz"

## AYLIEN credentials
application_id = "4b85d18c"
application_key = "07a17f39487276de81dafe3114335355"

## set up an instance of Tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

## set up an instance of the AYLIEN Text API
client = textapi.Client(application_id, application_key)

## search Twitter for something that interests you
number = input("How many Tweets do you want to analyze? \n")
userName = input("What user would you like to search for? \n")

user = api.get_user(userName)

results = api.search(
   lang="en",
   q="$" + userName+ " -rt",
   count=number,
   result_type="recent"
)


systime = time.strftime("%Y%m%d")

print("--- Gathered Tweets \n")

## open a csv file to store the Tweets and their sentiment 
file_name = 'Sentiment_Analysis_of_{}_Tweets_About_{}_{}.csv'.format(number, userName,systime)

with open(file_name, 'w', newline='') as csvfile:
   csv_writer = csv.DictWriter(
       f=csvfile,
       fieldnames=["Time","Tweet", "Sentiment"]
   )
   csv_writer.writeheader()

   print("--- Opened a CSV file to store the results of your sentiment analysis... \n")

## tidy up the Tweets and send each to the AYLIEN Text API
   for c, result in enumerate(results, start=1):
       tweet = result.text
       tweettime = result.created_at
       print(tweettime)
       tidy_tweet = tweet.strip().encode('ascii', 'ignore')

       if len(tweet) == 0:
           print('Empty Tweet')
           continue

       response = client.Sentiment({'text': tidy_tweet})
       csv_writer.writerow({
           'Time' : tweettime,
           'Tweet': response['text'],
           'Sentiment': response['polarity']
           
       })

       print("Analyzed Tweet {}".format(c))

## count the data in the Sentiment column of the CSV file 
with open(file_name, 'r') as data:
   counter = Counter()
   for row in csv.DictReader(data):
       counter[row['Sentiment']] += 1

   positive = counter['positive']
   negative = counter['negative']
   neutral = counter['neutral']

## declare the variables for the pie chart, using the Counter variables for "sizes"
colors = ['green', 'red', 'grey']
sizes = [positive, negative, neutral]
labels = 'Positive', 'Negative', 'Neutral'

## use matplotlib to plot the chart
plt.pie(
   x=sizes,
   shadow=True,
   colors=colors,
   labels=labels,
   startangle=90
)

plt.title("Sentiment of {} Tweets about {} from {}".format(number, userName,tweettime))
plt.show()

How many Tweets do you want to analyze? 
70
What user would you like to search for? 
AAPL
--- Gathered Tweets 

--- Opened a CSV file to store the results of your sentiment analysis... 

2018-04-21 17:25:18
Analyzed Tweet 1
2018-04-21 17:22:29
Analyzed Tweet 2
2018-04-21 17:18:01
Analyzed Tweet 3
2018-04-21 17:15:08
Analyzed Tweet 4
2018-04-21 17:04:42
Analyzed Tweet 5
2018-04-21 17:02:39
Analyzed Tweet 6
2018-04-21 17:01:51
Analyzed Tweet 7
2018-04-21 17:00:42
Analyzed Tweet 8
2018-04-21 16:54:30
Analyzed Tweet 9
2018-04-21 16:40:25
Analyzed Tweet 10
2018-04-21 16:37:04
Analyzed Tweet 11
2018-04-21 16:36:51
Analyzed Tweet 12
2018-04-21 16:33:26
Analyzed Tweet 13
2018-04-21 16:30:13
Analyzed Tweet 14
2018-04-21 16:28:30
Analyzed Tweet 15
2018-04-21 16:25:42
Analyzed Tweet 16
2018-04-21 16:21:02
Analyzed Tweet 17
2018-04-21 16:14:08
Analyzed Tweet 18
2018-04-21 16:11:12
Analyzed Tweet 19
2018-04-21 16:08:39
Analyzed Tweet 20
2018-04-21 16:07:53
Analyzed Tweet 21
2018-04-21 16:07:49
Anal

HttpError: <HttpError 429 when requesting https://api.aylien.com/api/v1/sentiment returned "Too Many Requests">

### The tweets are collected in a csv in the same folder with columns as Tweet, Sentiment and datetime of tweet when it was created. The time is server time